In [ ]:
import cv2
import numpy as np
import networkx as nx
from scipy.optimize import linear_sum_assignment
import math

def extract_graph_from_image(image_path, min_area=40, threshold_dist=150):
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Failed to load image: {image_path}")
    
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # === Step 1: Detect red areas (rooms) ===
    lower_red1 = np.array([0, 50, 30])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([160, 50, 30])
    upper_red2 = np.array([180, 255, 255])
    mask_red = cv2.inRange(hsv, lower_red1, upper_red1) | cv2.inRange(hsv, lower_red2, upper_red2)

    red_kernel = np.ones((3, 3), np.uint8)
    mask_red = cv2.morphologyEx(mask_red, cv2.MORPH_CLOSE, red_kernel, iterations=0)

    # === Step 2: Black line detection (refined) ===
    mask_black_thresh = cv2.adaptiveThreshold(
        gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, blockSize=11, C=5
    )

    edges_strong = cv2.Canny(gray, 30, 100)
    edges_soft = cv2.Canny(gray, 10, 60)
    edges = cv2.bitwise_or(edges_strong, edges_soft)

    mask_black = cv2.bitwise_or(mask_black_thresh, edges)

    black_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    mask_black = cv2.morphologyEx(mask_black, cv2.MORPH_CLOSE, black_kernel, iterations=1)

    # === Step 3: Subtract black outlines from red areas ===
    mask_cleaned = cv2.bitwise_and(mask_red, cv2.bitwise_not(mask_black))

    # === Step 4: Connected components ===
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(mask_cleaned, connectivity=8)

    G = nx.Graph()
    room_centroids = []

    for i in range(1, num_labels):  # Skip background
        area = stats[i, cv2.CC_STAT_AREA]
        if area >= min_area:
            cx, cy = centroids[i]
            room_centroids.append((int(cx), int(cy)))
            G.add_node(len(room_centroids) - 1, pos=(int(cx), int(cy)))

    for i, (x1, y1) in enumerate(room_centroids):
        for j, (x2, y2) in enumerate(room_centroids):
            if i < j:
                dist = np.linalg.norm([x1 - x2, y1 - y2])
                if dist < threshold_dist:
                    G.add_edge(i, j)

    print(f"{image_path} → Detected {len(G.nodes)} rooms (nodes) and {len(G.edges)} edges")
    return G

def compute_orientation_matrix(G):
    pos = nx.get_node_attributes(G, 'pos')
    nodes = list(G.nodes)
    N = len(nodes)
    theta = np.zeros((N, N))

    for i in range(N):
        x1, y1 = pos[nodes[i]]
        for j in range(N):
            if i != j:
                x2, y2 = pos[nodes[j]]
                dx, dy = x2 - x1, y2 - y1
                angle = np.degrees(np.arctan2(dy, dx)) % 360
                theta[i, j] = angle
            else:
                theta[i, j] = np.nan
    return theta

def compare_orientation_matrices_with_matching(G1, G2):
    pos1 = nx.get_node_attributes(G1, 'pos')
    pos2 = nx.get_node_attributes(G2, 'pos')
    nodes1 = list(G1.nodes)
    nodes2 = list(G2.nodes)

    if not pos1 or not pos2 or len(nodes1) == 0 or len(nodes2) == 0:
        print("Error: One or both graphs have no nodes.")
        return float('nan')

    coords1 = np.array([pos1[n] for n in nodes1])
    coords2 = np.array([pos2[n] for n in nodes2])

    dist_matrix = np.linalg.norm(coords1[:, None, :] - coords2[None, :, :], axis=2)
    row_ind, col_ind = linear_sum_assignment(dist_matrix)

    matched_nodes1 = [nodes1[i] for i in row_ind]
    matched_nodes2 = [nodes2[j] for j in col_ind]

    orient1 = compute_orientation_matrix(G1)
    orient2 = compute_orientation_matrix(G2)

    N = len(row_ind)
    theta1_subset = np.full((N, N), np.nan)
    theta2_subset = np.full((N, N), np.nan)

    for i in range(N):
        for j in range(N):
            if i != j:
                theta1_subset[i, j] = orient1[matched_nodes1[i], matched_nodes1[j]]
                theta2_subset[i, j] = orient2[matched_nodes2[i], matched_nodes2[j]]

    diff = np.abs(theta1_subset - theta2_subset)
    diff = np.nanmin([diff, 360 - diff], axis=0)
    rmse = np.sqrt(np.nanmean(diff**2))
    return rmse

def compute_ged(G1, G2, timeout=10):
    return nx.graph_edit_distance(G1, G2, timeout=timeout)

# === Example Usage ===

original_path = r #copy path of original floor plan
sketch_path = r #copy path of sketched floor plan

G_original = extract_graph_from_image(original_path)
G_sketch = extract_graph_from_image(sketch_path)

ged_score = compute_ged(G_original, G_sketch)
rmse_score = compare_orientation_matrices_with_matching(G_original, G_sketch)

print(f"Graph Edit Distance: {ged_score}")
print(f"Graph-based Orientation RMSE: {rmse_score:.2f}°")


C:\Users\DELL\OneDrive - University of Glasgow\Documents\Project\Edited_Data\Oculus_Rift\Rotational_Coloured\Rotational_Original.jpg → Detected 18 rooms (nodes) and 59 edges
C:\Users\DELL\OneDrive - University of Glasgow\Documents\Project\Edited_Data\Oculus_Rift\Rotational_Coloured\RS012_map.jpg → Detected 17 rooms (nodes) and 30 edges
Graph Edit Distance: 36.0
Graph-based Orientation RMSE: 6.76°


C:\Users\DELL\AppData\Local\Temp\ipykernel_16436\1864108735.py:116: RuntimeWarning: All-NaN axis encountered
  diff = np.nanmin([diff, 360 - diff], axis=0)
